# Predicción 

Haremos una predicción de la variable `Mas_1_Coche` en un nuevo set de datos empleando un modelo ya entrenado de XGBoost.

El primer paso es importar las librerías necesarias.

In [239]:
# Cargamos las librerías necesarias.
import pandas as pd
import pickle
import joblib
from sklearn.preprocessing import StandardScaler

Ahora cargamos nuestros datos.

In [240]:
# Cargamos nuestros datos.
X = pd.read_csv('../data/car_data_input_clean.csv', sep=',')

# Mostramos la forma y los primeros registros de nuestro DataFrame.
print(X.shape)
X.head()

(23199, 21)


,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,Campanna2,Campanna3,Zona _Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Revisiones,Edad Cliente
0,0,0,0,1,1,0,0,1,1,4,...,0,0,2,0,2,1,3,0,2,0
1,0,0,0,1,1,0,0,0,1,47,...,0,0,2,1,3,0,0,0,2,2
2,0,0,0,1,1,3,0,1,1,30,...,0,0,1,0,3,0,0,0,4,0
3,0,0,0,1,1,2,0,0,2,41,...,0,1,0,0,3,0,2,0,3,0
4,0,0,0,1,1,2,0,0,1,4,...,0,0,1,1,1,0,2,2,2,2


Cargamos el modelo con el que haremos la predicción y la variable `mapping` que contiene las equivalencias a las variables categóricas que modificamos a numéricas en la limpieza de datos.

In [241]:
# Importamos el modelo XGBoost y la variable de mapeo.

# Cargar el modelo.
modelo = joblib.load('../modelos/modelo_XGB.pkl')

# Cargar la variable.
with open('mappings.pkl', 'rb') as archivo:
    mappings = pickle.load(archivo)

Estandarizamos las columnas.

In [242]:
# Creamos un objeto StandardScaler.
scaler = StandardScaler()

# Obtenemos las columnas del DataFrame.
cols = X.columns

# Ajustamos y escalamos las columnas del DataFrame.
X[cols] = scaler.fit_transform(X[cols])

Hacemos predicciones y obtenemos probabilidades usando nuestro modelo de XGBoost.

In [243]:
# Hacer predicciones en el conjunto de prueba.
predictions = modelo.predict(X)

# También puedes obtener las probabilidades de las clases.
probabilities = modelo.predict_proba(X)

Agregamos estos resultados al DataFrame. De esta manera, podremos ver qué perfiles de clientes podrían comprar más de un coche según nuestra predicción.

In [244]:
# Agregar las predicciones y probabilidades al DataFrame.
X['Mas_1_Coche'] = predictions
X['Probabilidad_0'] = probabilities[:, 0]
X['Probabilidad_1'] = probabilities[:, 1]

# Mostrar el DataFrame con las predicciones y probabilidades.
X.head()

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Revisiones,Edad Cliente,Mas_1_Coche,Probabilidad_0,Probabilidad_1
0,-1.166854,-1.158413,-1.154338,-0.64256,0.307575,-0.913832,-0.53111,0.646688,-0.120832,-1.412864,...,-0.897815,-0.207286,1.289645,0.783597,-1.039652,-0.599505,-1.318503,0,0.913393,0.086607
1,-1.166854,-1.158413,-1.154338,-0.64256,0.307575,-0.913832,-0.53111,-1.546341,-0.120832,1.394932,...,1.113815,0.778333,-0.775407,-1.370869,-1.039652,-0.599505,0.738245,0,0.965275,0.034725
2,-1.166854,-1.158413,-1.154338,-0.64256,0.307575,1.533921,-0.53111,0.646688,-0.120832,0.284873,...,-0.897815,0.778333,-0.775407,-1.370869,-1.039652,0.196940,-1.318503,0,0.981233,0.018767
3,-1.166854,-1.158413,-1.154338,-0.64256,0.307575,0.718003,-0.53111,-1.546341,3.436510,1.003147,...,-0.897815,0.778333,-0.775407,0.065442,-1.039652,-0.201283,-1.318503,1,0.274681,0.725319
4,-1.166854,-1.158413,-1.154338,-0.64256,0.307575,0.718003,-0.53111,-1.546341,-0.120832,-1.412864,...,1.113815,-1.192906,-0.775407,0.065442,0.508465,-0.599505,0.738245,0,0.890533,0.109467


Deshacemos la estandarización de datos que hicimos antes de meterlos en nuestro modelo.

In [245]:
# Deshacer el escalado.
X_original = X.copy()  # Copiar el DataFrame original.
X_original[cols] = scaler.inverse_transform(X[cols])  # Deshacer el escalado.

# Mostrar el DataFrame tras deshacer el escalado.
X_original.head()

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Revisiones,Edad Cliente,Mas_1_Coche,Probabilidad_0,Probabilidad_1
0,-4.440892e-16,4.440892e-16,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,...,0.0,2.0,1.0,3.000000e+00,0.0,2.0,0.0,0,0.913393,0.086607
1,-4.440892e-16,4.440892e-16,0.0,1.0,1.0,0.0,0.0,0.0,1.0,47.0,...,1.0,3.0,0.0,-2.220446e-16,0.0,2.0,2.0,0,0.965275,0.034725
2,-4.440892e-16,4.440892e-16,0.0,1.0,1.0,3.0,0.0,1.0,1.0,30.0,...,0.0,3.0,0.0,-2.220446e-16,0.0,4.0,0.0,0,0.981233,0.018767
3,-4.440892e-16,4.440892e-16,0.0,1.0,1.0,2.0,0.0,0.0,2.0,41.0,...,0.0,3.0,0.0,2.000000e+00,0.0,3.0,0.0,1,0.274681,0.725319
4,-4.440892e-16,4.440892e-16,0.0,1.0,1.0,2.0,0.0,0.0,1.0,4.0,...,1.0,1.0,0.0,2.000000e+00,2.0,2.0,2.0,0,0.890533,0.109467


Redondeamos a valores enteros.

In [246]:
X_original[cols] = X_original[cols].round().astype(int)

# Mostrar el DataFrame tras redondear las columnas.
X_original.head()

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Revisiones,Edad Cliente,Mas_1_Coche,Probabilidad_0,Probabilidad_1
0,0,0,0,1,1,0,0,1,1,4,...,0,2,1,3,0,2,0,0,0.913393,0.086607
1,0,0,0,1,1,0,0,0,1,47,...,1,3,0,0,0,2,2,0,0.965275,0.034725
2,0,0,0,1,1,3,0,1,1,30,...,0,3,0,0,0,4,0,0,0.981233,0.018767
3,0,0,0,1,1,2,0,0,2,41,...,0,3,0,2,0,3,0,1,0.274681,0.725319
4,0,0,0,1,1,2,0,0,1,4,...,1,1,0,2,2,2,2,0,0.890533,0.109467


Una vez obtenemos los valores enteros, podemos aplicar el mapeo para tener nuestro DataFrame con variables categóricas.

In [247]:
# Invertir el diccionario de mapeo.
inverted_mappings = {col: {v: k for k, v in mapping.items()} for col, mapping in mappings.items()}

# Aplicar el mapeo invertido al DataFrame.
for column, mapping in inverted_mappings.items():
    X_original[column] = X_original[column].replace(mapping)

# Mostrar el DataFrame con las variables categóricas recuperadas.
X_original.head()

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Revisiones,Edad Cliente,Mas_1_Coche,Probabilidad_0,Probabilidad_1
0,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,Asturias,...,NO DATA,Averia muy grave,SI,3k-5k,0-10k,2,18-40,0,0.913393,0.086607
1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,Toledo,...,SI,No,NO,0-2k,0-10k,2,50-60,0,0.965275,0.034725
2,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,Lerida,...,NO DATA,No,NO,0-2k,0-10k,4,18-40,0,0.981233,0.018767
3,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,Santa Cruz de Tenerife,...,NO DATA,No,NO,2k-3k,0-10k,3,18-40,1,0.274681,0.725319
4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,Asturias,...,SI,Averia leve,NO,2k-3k,10k-20k,2,50-60,0,0.890533,0.109467


Podemos filtrar aquellos clientes que nuestro modelo predice que sí que van a comprar más de un coche.

In [248]:
# Filtrar los clientes predichos.
clientes_predichos = X_original[X_original['Mas_1_Coche'] == 1]

# Mostrar los clientes predichos.
clientes_predichos

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia_,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Revisiones,Edad Cliente,Mas_1_Coche,Probabilidad_0,Probabilidad_1
3,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,Santa Cruz de Tenerife,...,NO DATA,No,NO,2k-3k,0-10k,3,18-40,1,0.274681,0.725319
48,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,M,Empresa,Baleares,...,NO DATA,Averia muy grave,SI,0-2k,0-10k,1,18-40,1,0.368749,0.631251
62,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,M,Empresa,Baleares,...,NO DATA,Averia grave,NO,0-2k,0-10k,0,18-40,1,0.422111,0.577889
70,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,Sevilla,...,NO DATA,Averia muy grave,SI,5k-10k,0-10k,2,18-40,1,0.222666,0.777334
79,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,Guipuzcua,...,SI,No,NO,0-2k,0-10k,2,18-40,1,0.393713,0.606287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23174,I,TIPO7,FUEL 2,Alta,A,Financiera Marca,CASADO,M,Empresa,Barcelona,...,NO DATA,No,NO,2k-3k,0-10k,1,40-50,1,0.452266,0.547734
23178,I,TIPO7,FUEL 2,Alta,A,Financiera Marca,CASADO,M,Empresa,Valencia,...,NO DATA,No,NO,2k-3k,0-10k,1,40-50,1,0.141863,0.858137
23181,I,TIPO7,FUEL 1,Alta,A,Financiera Marca,CASADO,M,Empresa,Barcelona,...,NO DATA,No,NO,0-2k,0-10k,1,40-50,1,0.452266,0.547734
23182,I,TIPO7,FUEL 1,Alta,A,Financiera Marca,CASADO,M,Empresa,Madrid,...,SI,No,NO,2k-3k,10k-20k,1,50-60,1,0.373308,0.626692


Por último, guardamos los nuevos datos que contienen la información predicha.

In [249]:
# Guardar los resultados.
X_original.to_csv('../data/predicciones.csv', index=False)
clientes_predichos.to_csv('../data/clientes_predichos.csv', index=False)